In [1]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
import argparse
import os
import tensorflow as tf

import datetime as dt
import pandas as pd
import numpy as np
import regex as re
import random
import sys
import os


sys.path.append('./')

import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

# Logs Anomaly Detection 

Steps: 
- extract the log keys from raw logs (**log parsing**)
- analyse the log key exeuction path (**Model 1**)
- analyse the paramaters in log key  (**Model 2**)
- online learning for feedbacks

## Preprocessing

### Step1: Log parsing

In [14]:
import sys
sys.path.append('../')
from Drain import LogParser

input_dir  = 'data/'  # The input directory of log file
output_dir = 'Drain_result/'  # The output directory of parsing results
log_file   = 'Hadoop_2k'  # The input log file name
log_format = '<Date> <Time> <Pid> <Level> <Component>: <Content>'  # RtpDump log format
# Regular expression list for optional preprocessing (default: [])
regex      = [
    r'blk_(|-)[0-9]+' , # block id
    r'(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)', # IP
    r'(?<=[^A-Za-z0-9])(\-?\+?\d+)(?=[^A-Za-z0-9])|[0-9]+$', # Numbers
]
st         = 0.5  # Similarity threshold
depth      = 4  # Depth of all leaf nodes

parser = LogParser(log_format, indir=input_dir, outdir=output_dir,  depth=depth, st=st, rex=regex)

In [15]:
# parsing logs
%time parser.parse(log_file+'.log')

Parsing file: data/Hadoop_2k.log
Processed 50.0% of log lines.
Processed 100.0% of log lines.
Parsing done. [Time taken: 0:00:00.412834]
Wall time: 413 ms


In [66]:
#df = pd.read_csv('Drain_result/RtpDump_1.log_structured.csv')
df = pd.read_csv("Drain_result/data/Anomaly.csv")
#df.to_pickle('Drain_result/data/Anomaly.pkl')
df.head(5)

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,ParameterList
0,1,18/10/2015,"18:01:47,978",INFO,[main],org.apache.hadoop.mapreduce.v2.app.MRAppMaster,Created MRAppMaster for application appattempt...,19b75bb3,Created MRAppMaster for application appattempt...,"['1445144423722', '0020_000001']"
1,2,18/10/2015,"18:01:48,963",INFO,[main],org.apache.hadoop.mapreduce.v2.app.MRAppMaster,Executing with tokens:,3c5bb15c,Executing with tokens:,[]
2,3,18/10/2015,"18:01:48,963",INFO,[main],org.apache.hadoop.mapreduce.v2.app.MRAppMaster,"Kind: YARN_AM_RM_TOKEN, Service: , Ident: (app...",54d3cc6c,"Kind: YARN_AM_RM_TOKEN, Service: , Ident: (app...","['20', '1445144423722', '1', '-127633188']"
3,4,18/10/2015,"18:01:49,228",INFO,[main],org.apache.hadoop.mapreduce.v2.app.MRAppMaster,Using mapred newApiCommitter.,0594ecc2,Using mapred newApiCommitter.,[]
4,5,18/10/2015,"18:01:50,353",INFO,[main],org.apache.hadoop.mapreduce.v2.app.MRAppMaster,OutputCommitter set in config null,c0c8618d,OutputCommitter set in config null,[]


### Step 2: Create Batches (only if data is big)
This is the module to transform the clogs data to suitable format for whole anomaly detection
- Transforms CVS into text format
- Splits into batches data

this code gets @timestamp and message

In [20]:
import re
def csv_to_txt(inputfile, outputfile):
    # get the message parts from inputfile
    message_list, time_list, date_list = list(), list(), list()
    month, date, time = None, None, None
    print("we are prcoessing {}".format(inputfile))
    fd = pd.read_csv(inputfile)
    print('Data Loading done........ ')
    message_list = fd['Content']
    time_list = fd['Time']
    date_list = fd['Date'].str.replace(r'[\[\]]','') # removes list contents from date objects
    
    # match the message part 
    msg = r'(.*)"'        
    # match the date part
    date_pattern = r'(\d{2})-(\d{2})-(\d{4})'
    # match time
    time_pattern = r'(\d+:\d+:\d+)'
    
    with open(outputfile, 'w') as output_file:
        for message, time, date in zip(message_list, time_list, date_list):
            #print(f"the date is: {date} and time is {time}")
            message = message.strip()
            message = re.match(msg, message).group(1)
            month = re.match(date_pattern, date).group(2)
            date = re.match(date_pattern, date).group(1)
            time = re.match(time_pattern, time).group(1)
            timestamp = month+' '+ date +' '+time
            output_file.write(timestamp + ' ' + message+'\n')

Each batch contains 2000 samples

In [21]:
def batch_processing(filename, batch_dir):
    lines = []
    with open(filename, 'r+') as f:
        lines = f.readlines()

    length = len(lines)
    print("the length of txt is:", length)
    # define the batch number
    i = 0
    while i * 2000 <= length:
        with open(batch_dir+'batch'+'_'+ str(i) +'.txt', 'w') as fw:
            for row in lines[i * 2000 : (i+1) * 2000]:
                fw.write(row)
        i += 1

Define directories

In [22]:
input_dir = r'Drain_result/RtpDump_1.log_structured.csv' #input('Please input the path where the client_data lies in: ')
output_dir = 'data/' #input('Please input the path where the transformed client data lies in: ')
batch_dir = 'data/batches/' #input("Please input the path where the batched data lies in: ")

##### transform csv to txt file

In [23]:
csv_to_txt(input_dir, 'data/CSV_to_Txt.txt')

we are prcoessing Drain_result/RtpDump_1.log_structured.csv
Data Loading done........ 


<ipython-input-20-dc01b34a683a>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  date_list = fd['Date'].str.replace(r'[\[\]]','') # removes list contents from date objects


AttributeError: 'NoneType' object has no attribute 'group'

In [24]:
# creates batches 
batch_processing('data/CSV_to_Txt.txt', batch_dir)

the length of txt is: 1


FileNotFoundError: [Errno 2] No such file or directory: 'data/batches/batch_0.txt'

### Step 3: Data pre-processing functions
There are log message, log key, value_vector, parameter_value_vectors in the log dataframe
- 1. how to generate log message: time(seconds)---- first one should be computed especially + message part
- 2. log key ---- read the corresponding line from structured csv
- 3. parameter_value_vectors ---- decrease log key part with original message part

In [25]:
# import brew_distance
import difflib
from dateutil.parser import parse
# make it available to append a value to a key in a dict
from collections import defaultdict
from datetime import timedelta
import time

##### =========================== part to compute the log message ================================

In [26]:
def log_messages_create(fd):
    
    log_messages = []
    for month, date, time, message in zip(fd['Month'], fd['Day'], fd['Time'], fd['Content']):
        try:
            log_messages.append(str(month)+' '+str(date)+' '+str(time)+' '+ message)
        except Exception as e:
            pass
    return log_messages


##### =========================== part to compute the parameter value vectors =========================

In [27]:
# function to compute the time gap
def time_gap(time_lines):
    # input is the series of time lines
    # output is the difference between two sequent times
    times = []
    # calculate the gap between two time
    for j in range(len(time_lines) - 1):
        times.append((time_lines[j + 1] - time_lines[j]))
    times = list(times)
    # we set the first value of time gap as 0
    times.insert(0,0)

    return times

In [28]:
# function to convert the month, day, time into seconds
def trans_seconds(month_list, day_list, time_list):
    seconds_list = []
    seconds = 0
    for i in range(len(day_list)):
        # we assume there are 30 days for every month
        seconds = (month_list[i] - month_list[0]) * 30 * 24 * 3600 + (day_list[i] - day_list[0]) * 24 * 3600 + \
                  int(time_list[i][0]) * 3600 + int(time_list[i][1]) * 60 + int(time_list[i][2])
        # print("the seconds are:", seconds)
        seconds_list.append(seconds)
    return seconds_list

In [29]:
# function to get the variables (for parameter_value_vectors)
def show_diff(strings, strings1):
    # pattern = '[\ < \ * \ >, \(\ <.\ > \),:]'
    pattern = '[\<\*\>]'

    batch_variables = []
    for string, string1 in zip(strings, strings1):

        #print("we are comparing two string %s and %s"%(string, string1))
        # we should delete the <*> |:| (<*>) part in a log key

        string1 = re.sub(pattern,'', string1)
        # string - string1
        seqm = difflib.SequenceMatcher(None, string, string1)
        output = []
        # define the difference list
        variables =[]
        for opcode, a0, a1, b0, b1 in seqm.get_opcodes():
            if a0 == a1:
                pass
            if opcode == 'equal':
                pass
                #print("the sections [%d:%d] of string and [%d:%d] of string1 are the same" % (a0, a1, b0, b1))
            elif opcode == 'delete':
                #print("remove %s from positions [%d:%d]" % (string[a0:a1], a0, a1))
                output.append(seqm.a[a0:a1])

        batch_variables.append(output)
        #print("the batch_varibles are:",batch_variables)
    return batch_variables

#### Parameter Vector with time difference
- define the parameter value vector creation function
- we need use the variables in function single_match and append it after the time gap

- normally, we just analyse the system logs within one day, if we want to analyse logs beyond one day,
- we can extend the time process part

In [30]:
def parameter_vector(log_messages, log_keys, times, batch_variables, filename):
    '''
    :param log_messages: list of t + message
    :param log_keys: list of keys of raw logs
    :param times: list of time differences
    :param batch_variables: variables for the raw log and raw key
    :return: a dataframe with column names with raw_logs, log_keys, parameter_value_vectors
    '''

    parameter_value_vector = []
    parameter_value_vectors = []

    for time_v, variable in zip(times, batch_variables):
        #print('we are appending time:',time_v)
        # time.sleep(5)
        #print("we are appending variable:",variable)
        # time.sleep(5)
        parameter_value_vector = variable
        parameter_value_vector.insert(0,time_v)

        #print("the parameter_value_vector is:",parameter_value_vector)
        parameter_value_vectors.append(parameter_value_vector)

    # define the entry dict
    entry = {
        'log message': [],
        'log key': [],
        'parameter value vector': []
    }

    # transfer the entry to defaultdict
    entry = defaultdict(list)

    #print("the parameter_value_vectors are:",parameter_value_vectors)
    for raw_log, log_key, parameter_value_vector_v in zip(log_messages, log_keys, parameter_value_vectors):
        # assign values to every row
        # print("we are writing raw_log into log_message:", raw_log)
        entry['log message'].append(raw_log)
        # print("we are writing log_key into log_key", log_key)
        entry['log key'].append(log_key)
        # print("we are writing parameter_value_vector into dataframe:", parameter_value_vector_v)
        # time.sleep(5)
        entry['parameter value vector'].append(parameter_value_vector_v)

    fieldnames = ['log message', 'log key', 'parameter value vector']
    df = pd.DataFrame(entry, columns=fieldnames)
    #print("the entry is:", entry)
    # for normal linux system logs
    # df.to_csv('../data/System_logs/log_value_vector.csv', index=False)
    # for malicious linux system logs
    df.to_csv(filename, index=False)
    return df

.
.
.
.
.
## Main): load the structured_log csv

In [49]:
#structured_log_filename = f'Drain_result/{log_file}.log_structured.csv'
#df = pd.read_csv(structured_log_filename)
#df.dropna( inplace=True)
df

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,ParameterList,Month,Day
0,1,18/10/2015,18:01:47.978,INFO,[main],org.apache.hadoop.mapreduce.v2.app.MRAppMaster,Created MRAppMaster for application appattempt...,19b75bb3,Created MRAppMaster for application appattempt...,"['1445144423722', '0020_000001']",10,18
1,2,18/10/2015,18:01:48.963,INFO,[main],org.apache.hadoop.mapreduce.v2.app.MRAppMaster,Executing with tokens:,3c5bb15c,Executing with tokens:,[],10,18
2,3,18/10/2015,18:01:48.963,INFO,[main],org.apache.hadoop.mapreduce.v2.app.MRAppMaster,"Kind: YARN_AM_RM_TOKEN, Service: , Ident: (app...",54d3cc6c,"Kind: YARN_AM_RM_TOKEN, Service: , Ident: (app...","['20', '1445144423722', '1', '-127633188']",10,18
3,4,18/10/2015,18:01:49.228,INFO,[main],org.apache.hadoop.mapreduce.v2.app.MRAppMaster,Using mapred newApiCommitter.,0594ecc2,Using mapred newApiCommitter.,[],10,18
4,5,18/10/2015,18:01:50.353,INFO,[main],org.apache.hadoop.mapreduce.v2.app.MRAppMaster,OutputCommitter set in config null,c0c8618d,OutputCommitter set in config null,[],10,18
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,18/10/2015,18:10:54.202,WARN,[LeaseRenewer:msrabi@msra-sa-41:9000],org.apache.hadoop.ipc.Client,Address change detected. Old: msra-sa-41/10.19...,3a75edb9,Address change detected. Old: msra-sa-<*><*> N...,"['41/10.190.173.170:9000', '41:9000']",10,18
1996,1997,18/10/2015,18:10:54.202,WARN,[LeaseRenewer:msrabi@msra-sa-41:9000],org.apache.hadoop.hdfs.LeaseRenewer,Failed to renew lease for [DFSClient_NONMAPRED...,bb3261a3,Failed to renew lease for [DFSClient_NONMAPRED...,"['1537864556_1', '356']",10,18
1997,1998,18/10/2015,18:10:54.546,INFO,[RMCommunicator,Allocator] org.apache.hadoop.ipc.Client,Retrying connect to server: msra-sa-41:8030. A...,40363a14,Retrying connect to server: msra-sa-<*>:<*>. A...,"['41:8030', '0', '10', '1000']",10,18
1998,1999,18/10/2015,18:10:54.546,ERROR,[RMCommunicator,Allocator] org.apache.hadoop.mapreduce.v2.app....,ERROR IN CONTACTING RM.,b4ff2357,ERROR IN CONTACTING RM.,[],10,18


In [50]:
df[pd.isnull(df).any(axis=1)].count()

LineId           0
Date             0
Time             0
Pid              0
Level            0
Component        0
Content          0
EventId          0
EventTemplate    0
ParameterList    0
Month            0
Day              0
dtype: int64

####  Date Preprocessing

In [67]:
df['Date'] = df['Date'].str.replace(r'[\[\]\']','')
df['Month']= pd.to_datetime(df['Date']).dt.month
df['Day']= pd.to_datetime(df['Date']).dt.day
df['Time'] = df['Time'].apply(lambda x:  x.split(',', 1)[0])
#df['Time'] = pd.to_datetime(df['Time']).dt.time

df.head()

<ipython-input-67-d68b0057138b>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Date'] = df['Date'].str.replace(r'[\[\]\']','')
C:\Users\ashoaib\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '18/10/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\ashoaib\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '18/10/2015' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)


,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,ParameterList,Month,Day
0,1,18/10/2015,18:01:47,INFO,[main],org.apache.hadoop.mapreduce.v2.app.MRAppMaster,Created MRAppMaster for application appattempt...,19b75bb3,Created MRAppMaster for application appattempt...,"['1445144423722', '0020_000001']",10,18
1,2,18/10/2015,18:01:48,INFO,[main],org.apache.hadoop.mapreduce.v2.app.MRAppMaster,Executing with tokens:,3c5bb15c,Executing with tokens:,[],10,18
2,3,18/10/2015,18:01:48,INFO,[main],org.apache.hadoop.mapreduce.v2.app.MRAppMaster,"Kind: YARN_AM_RM_TOKEN, Service: , Ident: (app...",54d3cc6c,"Kind: YARN_AM_RM_TOKEN, Service: , Ident: (app...","['20', '1445144423722', '1', '-127633188']",10,18
3,4,18/10/2015,18:01:49,INFO,[main],org.apache.hadoop.mapreduce.v2.app.MRAppMaster,Using mapred newApiCommitter.,0594ecc2,Using mapred newApiCommitter.,[],10,18
4,5,18/10/2015,18:01:50,INFO,[main],org.apache.hadoop.mapreduce.v2.app.MRAppMaster,OutputCommitter set in config null,c0c8618d,OutputCommitter set in config null,[],10,18


In [68]:
# create the first column in dataframe with 'log message'
log_messages = log_messages_create(df)
log_keys = df['EventTemplate']
log_messages

['10 18 18:01:47 Created MRAppMaster for application appattempt_1445144423722_0020_000001',
 '10 18 18:01:48 Executing with tokens:',
 '10 18 18:01:48 Kind: YARN_AM_RM_TOKEN, Service: , Ident: (appAttemptId { application_id { id: 20 cluster_timestamp: 1445144423722 } attemptId: 1 } keyId: -127633188)',
 '10 18 18:01:49 Using mapred newApiCommitter.',
 '10 18 18:01:50 OutputCommitter set in config null',
 '10 18 18:01:50 OutputCommitter is org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter',
 '10 18 18:01:50 Registering class org.apache.hadoop.mapreduce.jobhistory.EventType for class org.apache.hadoop.mapreduce.jobhistory.JobHistoryEventHandler',
 '10 18 18:01:50 Registering class org.apache.hadoop.mapreduce.v2.app.job.event.JobEventType for class org.apache.hadoop.mapreduce.v2.app.MRAppMaster$JobEventDispatcher',
 '10 18 18:01:50 Registering class org.apache.hadoop.mapreduce.v2.app.job.event.TaskEventType for class org.apache.hadoop.mapreduce.v2.app.MRAppMaster$TaskEventDispatc

In [69]:
# append the timestamps to the log message

month_list, day_list, time_list = [], [], []
for i in range(len(df['Time'])):
    time_list.append(df['Time'][i].split(':'))
for j in range(len(df['Day'])):
    day_list.append(int(df['Day'][j]))
for j in range(len(df['Month'])):
    month_list.append(int(df['Month'][j]))

In [70]:
# call the generated month, day, time list to get the seconds
%time seconds_list = trans_seconds(month_list, day_list, time_list)

Wall time: 2.04 ms


In [71]:
# input the seconds_list into the time_gap functions to get the series of time difference
%time times = time_gap(seconds_list)

Wall time: 0 ns


In [72]:
import difflib
strings, strings1 = df['Content'], df['EventTemplate']

# generate the later values in parameter value vector
%time batch_variables = show_diff(strings=strings, strings1=strings1)

Wall time: 363 ms


In [73]:
print("the batch_variables are:", batch_variables[:3])

the batch_variables are: [['1445144423722', '0020', '000001'], [], ['20', '1445144423722', '1', '-127633188']]


In [74]:
# generate the final dataframe for vector anomaly detection 
# parameters with time difference

%time parameter_vector_df = parameter_vector(log_messages, log_keys, times, batch_variables, f'temp/{log_file}.log_value_vector_time_diff.csv')

Wall time: 23 ms


In [75]:
parameter_vector_df.head()

,log message,log key,parameter value vector
0,10 18 18:01:47 Created MRAppMaster for applica...,Created MRAppMaster for application appattempt...,"[0, 1445144423722, 0020, 000001]"
1,10 18 18:01:48 Executing with tokens:,Executing with tokens:,[1]
2,"10 18 18:01:48 Kind: YARN_AM_RM_TOKEN, Service...","Kind: YARN_AM_RM_TOKEN, Service: , Ident: (app...","[0, 20, 1445144423722, 1, -127633188]"
3,10 18 18:01:49 Using mapred newApiCommitter.,Using mapred newApiCommitter.,[1]
4,10 18 18:01:50 OutputCommitter set in config null,OutputCommitter set in config null,[1]


In [76]:
parameter_vector_df.to_csv('Drain_result/data/log_value_vector_anomaly.csv')

In [47]:
import regex as re
import numpy as np

lists = ['0,[8],[86]', '0,[8],[86]', '0,[8],[86]', '0,[4, 107, 108],[86]']


padd = max([len(re.findall('\d+',i)) for i in lists])
list_string = re.findall('\d+',lists[0])
list_string = [int(var) for var in list_string]
list_string = np.array(list_string)
np.pad(list_string, (0,padd-len(list_string)), 'constant')

array([ 0,  8, 86,  0,  0])

In [38]:
lists = ['0,[8],[86]', '0,[8],[86]', '0,[8],[86]', '0,[4, 107, 108],[86]']
[i.split() for i in lists]

[['0,[8],[86]'],
 ['0,[8],[86]'],
 ['0,[8],[86]'],
 ['0,[4,', '107,', '108],[86]']]